In [1]:
# 必要なパッケージをインポート
from pathlib import Path

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import obp
from obp.dataset import OpenBanditDataset
from obp.policy import IPWLearner
from obp.ope import (
    OffPolicyEvaluation, 
    RegressionModel,
    InverseProbabilityWeighting as IPS,
    DoublyRobust as DR
)

# Data Loading and Preprocessing

In [2]:
# ZOZOTOWNのトップページ推薦枠でBernoulli Thompson Sampling (bts)が収集したデータをダウンロードする
# `data_path=None`とすると、スモールサイズのお試しデータセットを用いることができる
dataset = OpenBanditDataset(
    behavior_policy="bts", #データ収集に用いられた意思決定モデル
    campaign="men", #キャンペーン "men","women"
    data_path=Path("./open_bandit_dataset"), #データセットのパス
)

# デフォルトの前処理を施したデータを取得する
# タイムスタンプの前半70%をトレーニングデータ、後半30%をバリデーションデータとする
train_data, validation_data = dataset.obtain_batch_bandit_feedback(
    test_size=0.3,
    is_timeseries_split=True
)

train_data.keys()

/root/.cache/pypoetry/virtualenvs/mldesign-9TtSrW0h-py3.9/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


dict_keys(['n_rounds', 'n_actions', 'action', 'position', 'reward', 'pscore', 'context', 'action_context'])

In [3]:
# 行動（ファッションアイテム）の数
dataset.n_actions

34

In [4]:
# データ数
dataset.n_rounds

4077727

In [5]:
# デフォルトの前処理による特徴料の次元数
dataset.dim_context

27

In [6]:
# 推薦枠におけるポジションの数
dataset.len_list

3

# 意思決定モデルの学習

In [ ]:
# 内部で用いる分類器としてRandomForestを指定したIPWLearnerを定義する
new_decision_making_model = IPWLearner(
    n_actions=dataset.n_actions, #行動の数
    len_list=dataset.len_list, #推薦枠の数
    base_classifier=RandomForestClassifier(
        n_estimators=300, max_depth=10, min_samples_leaf=5, random_state=12345
    ),
)

#trainデータを用いて、意思決定モデルを学習する
new_decision_making_model.fit(
    context=train_data["context"], #特徴量(X_i)
    action=train_data["action"], #過去の意思決定モデルによる行動選択
    reward=train_data["reward"], #観測される目的変数
    position=train_data["position"], #行動が提示された推薦位置
    pscore=train_data["pscore"], #過去の意思決定モデルによる行動選択確率
)

#validationデータに対して行動を選択
action_dist = new_decision_making_model.predict(
    context=validation_data["context"]
)

# 意思決定モデルの性能評価

In [ ]:
# DR推定量を用いるのに必要な目的変数予測モデルを得る
# opeモジュールに実装されている`RegressionModel`に好みの機械学習手法を与えば良い
regression_model = RegressionModel(
    n_actions_dataset.n_actions, #行動の数
    len_list=dataset.len_list, #推薦枠内のポジション数
    base_model=LogisticRegression(C=100, max_iter=10000, random_state=12345)
)

# `fit_predict`メソッドにより、バリデーションデータにおける期待報酬を推定
estimated_rewards_by_reg_model = regression_model.fit_predict(
    context=validation_data["context"], #特徴量(X_i)
    action=validation_data["action"], #過去の意思決定モデルによる行動選択
    reward=validation_data["reward"], #観測される目的変数
    position=validation_data["position"], #行動が提示された推薦位置
    random_state=12345
)

In [ ]:
# 意思決定モデルの性能評価を一気通貫で行うための`OffPolicyEvaluation`を定義する
ope = OffPolicyEvaluation(
    bandit_feedback=validation_data, #バリデーションデータ
    ope_estimators=[IPS(), DR()] #使用する推定量
)

In [ ]:
# 内部で用いる分類器としてロジスティック回帰を指定したIPWLearnerの性能をOPEにより評価
ope.visualize_off_policy_estimates(
    action_dist=action_dist, #evaluation_policy_aによるバリデーションデータに対する行動選択
    estimated_rewards_by_reg_model=estimated_rewards_by_reg_model,
    is_relative=True, #過去の意思決定モデルの性能に対する相対的な改善率を出力
    random_state=12345
)